<a href="https://colab.research.google.com/github/ShirelShemesh/organizational_structure/blob/master/test_all_models_%2B_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb)

In [1]:
# i = []
# while(True):
#     i.append('a')

In [2]:
import json, os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_3344.json to spark_nlp_for_healthcare_spark_ocr_3344.json


In [3]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

     |████████████████████████████████| 212.4 MB 68 kB/s 
     |████████████████████████████████| 130 kB 51.9 MB/s 
     |████████████████████████████████| 198 kB 71.5 MB/s 
     |████████████████████████████████| 136 kB 25.3 MB/s 


In [4]:
import json
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.3.2
Spark NLP_JSL Version : 3.3.2


# Clinical NER Model Profiling Pretrained Pipeline

This pipeline can be used to explore all the available pretrained NER models at once. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with `embeddings_clinical`.

|Clinical NER Model List|
|-|
|ner_ade_clinical|
|ner_posology_greedy|
|ner_risk_factors|
|jsl_ner_wip_clinical|
|ner_human_phenotype_gene_clinical|
|jsl_ner_wip_greedy_clinical|
|ner_cellular|
|ner_cancer_genetics|
|jsl_ner_wip_modifier_clinical|
|ner_drugs_greedy|
|ner_deid_sd_large|
|ner_diseases|
|nerdl_tumour_demo|
|ner_deid_subentity_augmented|
|ner_jsl_enriched|
|ner_genetic_variants|
|ner_bionlp|
|ner_measurements_clinical|
|ner_diseases_large|
|ner_radiology|
|ner_deid_augmented|
|ner_anatomy|
|ner_chemprot_clinical|
|ner_posology_experimental|
|ner_drugs|
|ner_deid_sd|
|ner_posology_large|
|ner_deid_large|
|ner_posology|
|ner_deidentify_dl|
|ner_deid_enriched|
|ner_bacterial_species|
|ner_drugs_large|
|ner_clinical_large|
|jsl_rd_ner_wip_greedy_clinical|
|ner_medmentions_coarse|
|ner_radiology_wip_clinical|
|ner_clinical|
|ner_chemicals|
|ner_deid_synthetic|
|ner_events_clinical|
|ner_posology_small|
|ner_anatomy_coarse|
|ner_human_phenotype_go_clinical|
|ner_jsl_slim|
|ner_jsl|
|ner_jsl_greedy|
|ner_events_admission_clinical|

You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [5]:
from sparknlp.pretrained import PretrainedPipeline

clinical_profiling_pipeline = PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")



ner_profiling_clinical download started this may take some time.
Approx size to download 2.3 GB
[OK!]


In [6]:
text = """67 year old with metastatic breast cancer, COPD (on chronic prednisone and home O2), 
            paroxysmal afib who presents with worsening SOB and some confusion. 
            She has had malignant left pleural effusion that was last drained on February 10.
            She is taking tamoxifen for her breast cancer. She was last admitted on 02/02/2018 and discharged with home hospice/palliative care. 
            In terms of her cardiac issues, she was reportedly diagnosed with afib in 2011. 
            She had an echo at that time showing normal LV function. Most recently, she had an echo November 2012 showing EF 35-40%. 
            It does not appear that she underwent any ischemia evaluation."""

In [7]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab, hcc=False):
    
    full_light_result = lp.fullAnnotate(text)

    codes = []
    distances =[]
    resolutions=[]

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
        codes.append(code.result) 
        distances.append(code.metadata['all_k_distances'].split(':::')[0])
        resolutions.append(code.metadata['all_k_resolutions'].split(':::')[0])

    distance = 'distance:', distances
    
    return vocab, codes, distance, resolutions

In [8]:
clinical_result = clinical_profiling_pipeline.annotate(text)
clinical_result.keys()

dict_keys(['ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_posology_greedy_chunks', 'ner_radiology_wip_clinical', 'ner_deidentify_dl', 'ner_jsl_slim', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_deid_synthetic', 'ner_drugs_greedy', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greedy', 'ner_cellular_chunks', 'ner_cancer_genetics_chunks', 'ner_jsl_greedy', 'jsl_ner_wip_modifier_clinical_chunks', 'ner_drugs_greedy_chunks', 'ner_deid_sd_large_chunks', 'ner_diseases_chunks', 'ner_diseases_large', 'ner_chemprot_clinical', 'ner_posology_large', 'nerdl_tumour_demo_chunks', 'ner_deid_subentity_augmented_chunks', 'ner_jsl_enriched_chunks', 'ner_genetic_variants_chunks', 'ner_chexpert', 'ner_bionlp_chunks', 'ner_measurements_clinical_chunks', 'ner_diseases_large_chunks', 'ner_drugs_large', 'ner_clinical_large', 'ner_chemicals', 'ner_radiology_chunks', 'ner_bacterial_species', 'ner_deid_aug

In [9]:
print("Clinical Text: \n", text, "\n \nResults:\n")

# for i in clinical_result.keys():
#   print("{} : ".format(i), clinical_result[i])

print("{} : ".format("ner_drugs_greedy_chunks"), clinical_result["ner_drugs_greedy_chunks"])

Clinical Text: 
 67 year old with metastatic breast cancer, COPD (on chronic prednisone and home O2), 
            paroxysmal afib who presents with worsening SOB and some confusion. 
            She has had malignant left pleural effusion that was last drained on February 10.
            She is taking tamoxifen for her breast cancer. She was last admitted on 02/02/2018 and discharged with home hospice/palliative care. 
            In terms of her cardiac issues, she was reportedly diagnosed with afib in 2011. 
            She had an echo at that time showing normal LV function. Most recently, she had an echo November 2012 showing EF 35-40%. 
            It does not appear that she underwent any ischemia evaluation. 
 
Results:

ner_drugs_greedy_chunks :  ['prednisone', 'tamoxifen']


In [10]:
def isBlank (myString):
    return not (myString and myString.strip() and not myString == ".")

In [11]:
# initialize assembler & embedder for entity resolving 
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [12]:
def get_rxNorm_code(text):
      
  print('getting code for ', text)
  rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

  rxnorm_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])
  
  rxnorm_lp = LightPipeline(rxnorm_pipelineModel)

  code = get_codes (rxnorm_lp, text, vocab='rxnorm_code')
  print('code for ', text, 'is: ', code)
  return(code)

In [13]:
def get_snomed_code(text):
      
  print('getting code for ', text)
  snomed_ct_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings_aux_concepts","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

  snomed_pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          sbert_embedder,
          snomed_ct_resolver])

  snomed_lp = LightPipeline(snomed_pipelineModel)


  code = get_codes (snomed_lp, text, vocab='snomed_code', aux_label=True)
  print('code for ', text, 'is: ', code)
  return(code)

In [14]:
def get_icd10_code(text):
      
  icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
        .setInputCols(["ner_chunk", "sbert_embeddings"]) \
        .setOutputCol("icd10cm_code")\
        .setDistanceFunction("EUCLIDEAN")

  icd_pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          sbert_embedder,
          icd10_resolver])

  icd_lp = LightPipeline(icd_pipelineModel)

  return get_codes (icd_lp, text, vocab='rxnorm_code')

In [15]:
def get_rxnorm_resolution(text):
  print('resolving rxNorm resolution for text:', text)
  results = {'rxnorm code':get_rxNorm_code(text)}
  print(results)
  return results



In [16]:
def get_snomed_resolution(text):
  print('resolving snomed resolution for text:', text)
  results = {'snomed code':get_snomed_code(text)}
  print(results)
  return results



In [17]:
# def init_clinical_RE_model():
#   documenter = DocumentAssembler()\
#     .setInputCol("text")\
#     .setOutputCol("document")

#   sentencer = SentenceDetector()\
#       .setInputCols(["document"])\
#       .setOutputCol("sentences")

#   tokenizer = sparknlp.annotators.Tokenizer()\
#       .setInputCols(["sentences"])\
#       .setOutputCol("tokens")

#   words_embedder = WordEmbeddingsModel()\
#       .pretrained("embeddings_clinical", "en", "clinical/models")\
#       .setInputCols(["sentences", "tokens"])\
#       .setOutputCol("embeddings")

#   pos_tagger = PerceptronModel()\
#       .pretrained("pos_clinical", "en", "clinical/models") \
#       .setInputCols(["sentences", "tokens"])\
#       .setOutputCol("pos_tags")

#   events_admission_ner_tagger = MedicalNerModel()\
#       .pretrained("ner_events_admission_clinical", "en", "clinical/models")\
#       .setInputCols("sentences", "tokens", "embeddings")\
#       .setOutputCol("ner_tags")   

#   ner_chunker = NerConverterInternal()\
#       .setInputCols(["sentences", "tokens", "ner_tags"])\
#       .setOutputCol("ner_chunk")

#   dependency_parser = DependencyParserModel()\
#       .pretrained("dependency_conllu", "en")\
#       .setInputCols(["sentences", "pos_tags", "tokens"])\
#       .setOutputCol("dependencies")

#   clinical_re_Model = RelationExtractionModel()\
#       .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
#       .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
#       .setOutputCol("relations")\
#       .setMaxSyntacticDistance(4)\
#       .setPredictionThreshold(0.9)

#   pipeline = Pipeline(stages=[
#       documenter,
#       sentencer,
#       tokenizer, 
#       words_embedder, 
#       pos_tagger, 
#       events_admission_ner_tagger,
#       ner_chunker,
#       dependency_parser,
#       clinical_re_Model
#   ])


#   empty_data = spark.createDataFrame([[""]]).toDF("text")

#   return pipeline.fit(empty_data)

In [18]:
# import pandas as pd

# def get_clinical_assertion_light (light_model, text):

#   light_result = light_model.fullAnnotate(text)[0]

#   status=[]

#   for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
#       status.append(m.result)

#   return status

import pandas as pd

def get_clinical_assertion_light (light_result):

  chunks=[]
  entities=[]
  status=[]

  print('light result to assert is:',light_result)

  # light_result = light_result
  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)

  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

  return df

**Lets show chunk results of NER models in a pandas dataframe.**

In [19]:
def get_relations_df (results, col='relations'):
  print('getting relations for:',results)
  rel_pairs=[]
  for rel in results[0][col]:
    rel_pairs.append(dict(
        result=rel.result, 
        entity1=rel.metadata['entity1'], 
        entity1_begin=rel.metadata['entity1_begin'],
        entity1_end=rel.metadata['entity1_end'],
        chunk1=rel.metadata['chunk1'], 
        entity2=rel.metadata['entity2'],
        entity2_begin=rel.metadata['entity2_begin'],
        entity2_end=rel.metadata['entity2_end'],
        chunk2=rel.metadata['chunk2'], 
        confidence=rel.metadata['confidence']
    ))

  return rel_pairs

In [20]:
 from sparknlp_jsl.annotator import *
def get_assertions(text, pipeline):

  print('trying to assert...')
  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = pipeline.fit(empty_data)
  light_model = LightPipeline(model)


  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]

  print('assertion:',light_result)

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)
          
  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

  return df

In [21]:
def get_ner_drugs_pipeline():
  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  clinical_ner = MedicalNerModel\
      .pretrained("ner_drugs_greedy", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")\
      .setIncludeAllConfidenceScores(True)

  ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")


  pos_tagger = PerceptronModel()\
      .pretrained("pos_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token"])\
      .setOutputCol("pos_tags")

  events_admission_ner_tagger = MedicalNerModel()\
      .pretrained("ner_drugs_greedy", "en", "clinical/models")\
      .setInputCols("sentence", "token", "embeddings")\
      .setOutputCol("ner_tags")   

  ner_chunker = NerConverterInternal()\
      .setInputCols(["sentence", "token", "ner_tags"])\
      .setOutputCol("ner_chunk")

  dependency_parser = DependencyParserModel()\
      .pretrained("dependency_conllu", "en")\
      .setInputCols(["sentence", "pos_tags", "token"])\
      .setOutputCol("dependencies")

  clinical_re_Model = RelationExtractionModel()\
      .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
      .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
      .setOutputCol("relations")\
      .setMaxSyntacticDistance(4)\
      .setPredictionThreshold(0.9)


  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
  clinical_assertion = AssertionDLModel.pretrained("assertion_dl_healthcare", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")
      
  
  pipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer, 
      word_embeddings, 
      clinical_ner,
      ner_converter,
      clinical_assertion,
      pos_tagger, 
      events_admission_ner_tagger,
      ner_chunker,
      dependency_parser,
      clinical_re_Model
  ])

  return pipeline


In [22]:
def get_ner_diseases_pipeline():
  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  clinical_ner = MedicalNerModel\
      .pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")\
      .setIncludeAllConfidenceScores(True)

  ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")


  pos_tagger = PerceptronModel()\
      .pretrained("pos_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token"])\
      .setOutputCol("pos_tags")

  events_admission_ner_tagger = MedicalNerModel()\
      .pretrained("jsl_ner_wip_clinical", "en", "clinical/models")\
      .setInputCols("sentence", "token", "embeddings")\
      .setOutputCol("ner_tags")   

  ner_chunker = NerConverterInternal()\
      .setInputCols(["sentence", "token", "ner_tags"])\
      .setOutputCol("ner_chunks")

  dependency_parser = DependencyParserModel()\
      .pretrained("dependency_conllu", "en")\
      .setInputCols(["sentence", "pos_tags", "token"])\
      .setOutputCol("dependencies")

  clinical_re_Model = RelationExtractionModel()\
      .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
      .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
      .setOutputCol("relations")\
      .setMaxSyntacticDistance(4)\
      .setPredictionThreshold(0.9)


  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
  clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")
      
  
  pipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer, 
      word_embeddings, 
      clinical_ner,
      ner_converter,
      clinical_assertion,
      pos_tagger, 
      events_admission_ner_tagger,
      ner_chunker,
      dependency_parser,
      clinical_re_Model
  ])

  return pipeline


In [23]:
def get_ner_posology_pipeline():
  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  clinical_ner = MedicalNerModel\
      .pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")\
      .setIncludeAllConfidenceScores(True)

  ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")


  pos_tagger = PerceptronModel()\
      .pretrained("pos_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token"])\
      .setOutputCol("pos_tags")

  events_admission_ner_tagger = MedicalNerModel()\
      .pretrained("ner_posology", "en", "clinical/models")\
      .setInputCols("sentence", "token", "embeddings")\
      .setOutputCol("ner_tags")   

  ner_chunker = NerConverterInternal()\
      .setInputCols(["sentence", "token", "ner_tags"])\
      .setOutputCol("ner_chunks")

  dependency_parser = DependencyParserModel()\
      .pretrained("dependency_conllu", "en")\
      .setInputCols(["sentence", "pos_tags", "token"])\
      .setOutputCol("dependencies")

  clinical_re_Model = RelationExtractionModel()\
      .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
      .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
      .setOutputCol("relations")\
      .setMaxSyntacticDistance(4)\
      .setPredictionThreshold(0.9)


  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
  clinical_assertion = AssertionDLModel.pretrained("assertion_jsl", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")
      
  
  pipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer, 
      word_embeddings, 
      clinical_ner,
      ner_converter,
      clinical_assertion,
      pos_tagger, 
      events_admission_ner_tagger,
      ner_chunker,
      dependency_parser,
      clinical_re_Model
  ])

  return pipeline


In [24]:
import pandas as pd


def get_drugs_chunk_result(original_text, light_result, model_name, pipeline):

  print('getting chunk results for light result:',light_result)
  
  all_relations = get_relations_extraction(original_text, pipeline)
  print('all relations for this text are:',all_relations)
  all_assertions = get_assertions(original_text, pipeline)
  print('all assertions for this text are:',all_assertions)

  sentences = []
  begins = []
  ends = []
  chunks = []
  entities = []
  confidences = []
  resolutions = []
  assertions = []
  relations = []
  # original_text_relations = []


  for n in (light_result):

    sentences.append(n.metadata['sentence'])
    begins.append(n.begin)
    ends.append(n.end) 
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidences.append(n.metadata['confidence'])
    resolutions.append(get_rxNorm_code(n.result))

    matching_relations = []
    

    for r in all_relations:
      if (('chunk1' in r and r['chunk1'] == n.result) or ('chunk2' in r and r['chunk2'] == n.result)):
        matching_relations.append(r)
    relations.append(matching_relations)


    matching_assertions = all_assertions.loc[all_assertions.chunks == n.result,'assertion']

    # assertions.append(all_assertions.loc[all_assertions['chunks'] == n.result]['assertion'])
    # assertions.append(all_assertions.loc[all_assertions.loc[all_assertions['chunks'] == n.result],'assertion'].values[0])
    # assertions.append(all_assertions.loc[all_assertions.chunks == n.result],'assertion'].values[0])
    if (len(matching_assertions) > 0):
      matching_assertions = matching_assertions.values[0]
    assertions.append(matching_assertions)

    # assertions.append(matching_assertions)
      # relations.append(all_relations.loc[all_relations['chunk1'] == n.result] or all_relations.loc[df['chunk2'] == n.result])
    
    # relations.append(all_relations.loc[all_relations['chunk1'] == n.result] or all_relations.loc[df['chunk2'] == n.result])
      
    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'chunks': chunks, 'entity': entities, 'confidence': confidences, 'resolution': resolutions, 'assertion': assertions, 'relations': relations})

    print("\n \n", "*"*20, model_name, "Model Results", "*"*20)
    
    display(df)

In [25]:
import pandas as pd


def get_diseases_chunk_result(original_text, light_result, model_name, pipeline):

  print('getting chunk results for light result:',light_result)
  
  # all_relations = get_relations_extraction(original_text, pipeline)
  # print('all relations for this text are:',all_relations)
  all_assertions = get_assertions(original_text, pipeline)
  print('all assertions for this text are:',all_assertions)

  sentences = []
  begins = []
  ends = []
  chunks = []
  entities = []
  confidences = []
  resolutions = []
  assertions = []
  # relations = []
  # original_text_relations = []


  for n in (light_result):

    sentences.append(n.metadata['sentence'])
    begins.append(n.begin)
    ends.append(n.end) 
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidences.append(n.metadata['confidence'])
    resolutions.append(get_snomed_code(n.result))

    matching_relations = []
    

    # for r in all_relations:
    #   if (('chunk1' in r and r['chunk1'] == n.result) or ('chunk2' in r and r['chunk2'] == n.result)):
    #     matching_relations.append(r)
    # relations.append(matching_relations)


    matching_assertions = all_assertions.loc[all_assertions.chunks == n.result,'assertion']

    # assertions.append(all_assertions.loc[all_assertions['chunks'] == n.result]['assertion'])
    # assertions.append(all_assertions.loc[all_assertions.loc[all_assertions['chunks'] == n.result],'assertion'].values[0])
    # assertions.append(all_assertions.loc[all_assertions.chunks == n.result],'assertion'].values[0])
    if (len(matching_assertions) > 0):
      matching_assertions = matching_assertions.values[0]
    assertions.append(matching_assertions)

    # assertions.append(matching_assertions)
      # relations.append(all_relations.loc[all_relations['chunk1'] == n.result] or all_relations.loc[df['chunk2'] == n.result])
    
    # relations.append(all_relations.loc[all_relations['chunk1'] == n.result] or all_relations.loc[df['chunk2'] == n.result])
      
    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'chunks': chunks, 'entity': entities, 'confidence': confidences, 'resolution': resolutions, 'assertion': assertions 
                      #  , 'relations': relations
                       })

    print("\n \n", "*"*20, model_name, "Model Results", "*"*20)
    
    display(df)

In [26]:
text2 = """
 67 year old with metastatic breast cancer, COPD (on chronic prednisone and home O2), 
            paroxysmal afib who presents with worsening SOB and some confusion. 
            She has had malignant left pleural effusion that was last drained DATE[Feb 10].
            She is taking tamoxifen for her breast cancer. She was last admitted on 02/02/2018 and discharged with home hospice/palliative care. 
            In terms of her cardiac issues, she was reportedly diagnosed with afib in 2011. 
            She had an echo at that time showing normal LV function. Most recently, she had an echo DATE[Nov 2012] showing EF 35-40%. 
            It does not appear that she underwent any ischemia evaluation. 
"""

In [27]:
clinical_full_result = clinical_profiling_pipeline.fullAnnotate(text2)[0]

In [28]:
def get_relations_extraction(text, pipeline):
  
  print('Will now try to extract relations for:',text)
  
  empty_data = spark.createDataFrame([[""]]).toDF("text")
  model = pipeline.fit(empty_data)

  light_model = LightPipeline(model)
  annotations = light_model.fullAnnotate(text)

  print('annotations are:', annotations)

  rel_df = get_relations_df (annotations)
  print('rel_df is:',rel_df)

  return rel_df

In [29]:
clinical_full_result['ner_drugs_greedy_chunks']

[Annotation(chunk, 62, 71, prednisone, {'entity': 'DRUG', 'sentence': '0', 'chunk': '0', 'confidence': '0.7363'}),
 Annotation(chunk, 287, 295, tamoxifen, {'entity': 'DRUG', 'sentence': '2', 'chunk': '1', 'confidence': '0.9987'})]

In [30]:
print("Clinical Text:\n", text2)

Clinical Text:
 
 67 year old with metastatic breast cancer, COPD (on chronic prednisone and home O2), 
            paroxysmal afib who presents with worsening SOB and some confusion. 
            She has had malignant left pleural effusion that was last drained DATE[Feb 10].
            She is taking tamoxifen for her breast cancer. She was last admitted on 02/02/2018 and discharged with home hospice/palliative care. 
            In terms of her cardiac issues, she was reportedly diagnosed with afib in 2011. 
            She had an echo at that time showing normal LV function. Most recently, she had an echo DATE[Nov 2012] showing EF 35-40%. 
            It does not appear that she underwent any ischemia evaluation. 



In [31]:
# ner_posology_pipeline = get_ner_posology_pipeline()
# ner_posology_chunks_result = clinical_full_result['ner_posology_chunks']

# print(ner_posology_chunks_result)
# get_drugs_chunk_result(text, ner_posology_chunks_result, 'ner_posology_chunks', ner_posology_pipeline)

In [32]:
# ner_drugs_chunks_result = clinical_full_result['ner_drugs_greedy_chunks']
# pipeline = get_ner_drugs_pipeline()
# print(ner_drugs_chunks_result)
# get_drugs_chunk_result(text, ner_drugs_chunks_result, 'ner_drugs_greedy_chunks', pipeline)

In [ ]:
ner_diseases_chunks_result = clinical_full_result['ner_diseases_chunks']
pipeline = get_ner_diseases_pipeline()
print(ner_diseases_chunks_result)
get_diseases_chunk_result(text, ner_diseases_chunks_result, 'ner_diseases_chunks', pipeline)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ \ ]

In [ ]:
text3_family_history = "Mother with a history of metastatic breast cancer."
ner_diseases_chunks_result = clinical_full_result['ner_diseases_chunks']
pipeline = get_ner_diseases_pipeline()
print(ner_diseases_chunks_result)
get_diseases_chunk_result(text3_family_history, ner_diseases_chunks_result, 'ner_diseases_chunks', pipeline)

In [ ]:
stage_t1 = """HPI:
From 3/11/2010 to April 2010 the patient received adjuvant chemotherapy (Taxol and carboplatin x 3 cycles). """
stage_t2 = """
HPI:
He was diagnosed with stage IV non-small cell lung cancer in Oct 1991 and has received cyberknife for brain lesions, carboplatin, pemetrexed, and PF-[**Telephone/Fax (3) 7932**] as part of a clinical trial.  He discontinued this trial [**Doctor Last Name 23**] on 2/92 because of clinical (worsening dyspena) and radiolographic evidence of cancer progression.   He then started a new [**Doctor Last Name 23**], Alimta, on 10-11-1993.
"""
stage_t3 = """
Sample Type / Medical Specialty:
Hematology - Oncology
Sample Name:
Consult - Breast Cancer
Description:
Patient presents with complaint of lump in the upper outer quadrant of the right breast
(Medical Transcription Sample Report)
CHIEF COMPLAINT / REASON FOR THE VISIT:
Patient has been diagnosed to have breast cancer.
BREAST CANCER HISTORY:
Patient presented with the following complaints: Lump in the upper outer quadrant of the right breast that has been present for the last 4 weeks. The lump is painless and the skin over the lump is normal. Patient denies any redness, warmth, edema and nipple discharge. Patient had a mammogram recently and was told to have a mass measuring 2 cm in the UOQ and of the left breast. Patient had an excisional biopsy of the mass and subsequently axillary nodal sampling.
PATHOLOGY:
Infiltrating ductal carcinoma, Estrogen receptor 56, Progesterone receptor 23, S-phase fraction 2., Her 2 neu 0 and all nodes negative.  
STAGE:
Stage I.
TNM STAGE:
T1, N0 and M0.
SURGERY:
S/P lumpectomy left breast and Left axillary node sampling. Patient is here for further recommendation.
PAST MEDICAL HISTORY:
Osteoarthritis for 5 years. ASHD for 10 years. Kidney stones recurrent for 10 years.
SCREENING TEST HISTORY:
Last rectal exam was done on 10/99. Last mammogram was done on 12/99. Last gynecological exam was done on 10/99. Last PAP smear was done on 10/99. Last chest x-ray was done on 10/99. Last F.O.B. was done on 10/99-X3. Last sigmoidoscopy was done on 1998. Last colonoscopy was done on 1996.
IMMUNIZATION HISTORY:
Last flu vaccine was given on 1999. Last pneumonia vaccine was given on 1996.
FAMILY MEDICAL HISTORY:
Father age 85, history of cerebrovascular accident (stroke) and hypertension. Mother history of CHF and emphysema that died at the age of 78. No brothers and sisters. 1 son healthy at age 54.
PAST SURGICAL HISTORY:
Appendectomy. Biopsy of the left breast 1996 - benign. Cholecystectomy.
PERSONAL AND SOCIAL HISTORY:
Marital status: Married. Smoking history: Smoked 1 PPD, quit 12 years ago and after smoking for 30 years. Alcohol history: Drinks socially. Denies any history of drug abuse.
ALLERGIES:
There are no known drug allergies.
CURRENT MEDICATIONS:
Aspirin 1 tab x 1 / day. Calan SR 120 mg. x 1 / day.
REVIEW OF SYSTEMS:
General: Patient feels fairly well. Patient denies history of fever, chills, night sweats and weight loss.
Head and Eyes: Patient denies any problems relating to the head and eyes.
Ears Nose and Throat: Patient has no problems related to the ears, nose or throat.
Respiratory: Patient denies any respiratory complaints, such as cough, shortness of breath, chest pain, wheezing, hemoptysis, etc.
Cardiovascular: Chest pain in the retrosternal area, Occasional anginal pain and patient describes it as a sensation of tightness. It radiates to the left shoulder. Patient denies any palpitation, syncope, paroxysmal nocturnal dyspnea and orthopnea.
Gastrointestinal: Patient denies any nausea, vomiting, abdominal pain, dysphagia or any altered bowel movements.
Genitourinary: Denies any genito-urinary complaints.
Musculoskeletal: The patient denies any musculoskeletal complaints.
Neurological: Patient denies any focal motor, sensory or other neurological symptoms.
PHYSICAL EXAMINATION:
General: Patient appears well developed, well nourished and healthy. Personality: pleasant and cooperative. Mental status: Alert and oriented. Stature: slender. ECOG performance score 0.
HEENT: Examination of head, eyes, ears, nose and throat is unremarkable.
Hematologic / Lymphatic: There is no palpable adenopathy in the inguinal, axillary, or cervical areas.
Cardiovascular: Heart: Regular rhythm, normal rate without any murmurs or gallops.
Breast: RIGHT BREAST: Within normal limits. LEFT BREAST: Consistency: slight induration noted due to recent surgery.
Respiratory: Chest symmetrical, normal, breath sounds equal, bilateral symmetrical, no rales or rhonchi and no
dullness to percussion.
Abdomen / Gastrointestinal: Abdomen is soft, non-tender, and without palpable masses. No hepatosplenomegaly is appreciable.
Extremities: Peripheral pulses are normal. There is no edema, cyanosis, clubbing or significant varicosities. No skin lesions identified.
Musculoskelatal: No evidence of joint swelling, bone tenderness or muscle tenderness is appreciable.
Neurological: Brief neurological examination reveals motor power grossly normal in all groups and no gross sensory or other abnormality appreciable.
RADIOLOGY:
Mammogram: A mass measuring 2X2 cm. in the upper outer quadrant of the left breast. Lab:
LAB DATA:
CMP (comprehensive metabolic panel): WNL. Liver function tests are WNL. CBC with diff shows WBC 3.2 / cmm. Hemoglobin 12.0 grams / dl, Platelets 250000 / cmm and it is dated 1/4/2000.
IMPRESSION / DIAGNOSIS
: Carcinoma of the left breast (174.9 - female), Upper outer quadrant (174.4)
PATHOLOGY:
Infiltrating ductal carcinoma. S/P lumpectomy and axillary node dissection. (Details as per HPI).
DISCUSSION:
Discussed in detail the diagnosis, prognosis and treatment alternatives. Options of treatment discussed. Side effects of Tamoxifen discussed in detail.
RECOMMENDATIONS:
Hormonal therapy with Tamoxifen and Radiation therapy to the breast is recommended.
TESTS ORDERED:
The following labs are to be drawn about a week or so prior to next appointment:
HEMATOLOGY: CBC.
CHEMISTRY: comprehensive metabolic panel (CMP) and liver function panel (LFT).
MEDICATIONS PRESCRIBED:
Nolvadex 20 mg. 1 time a day.
FOLLOW-UP INSTRUCTIONS:
Return to see William Smith.M.D. for follow up in 3 month (s). Make appointment to Radiation therapy.
"""


In [ ]:
ner_diseases_chunks_result = clinical_full_result['ner_diseases_chunks']
pipeline = get_ner_diseases_pipeline()
print(ner_diseases_chunks_result)
get_diseases_chunk_result(stage_t1, ner_diseases_chunks_result, 'ner_diseases_chunks', pipeline)

In [ ]:
# ner_posology_greedy_chunks_result = clinical_full_result['ner_posology_greedy_chunks']
# print(ner_posology_greedy_chunks_result)
# get_chunk_result(text, ner_posology_greedy_chunks_result, 'ner_posology_greedy_chunks')

**Now we will show all NER labels of tokens in the same dataframe.**

**cut** **here**

In [ ]:
import pandas as pd

def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.begin for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:
    
    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)
    
  return df

In [ ]:
# get_token_results(clinical_full_result)